# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier
import warnings
warnings.filterwarnings('ignore')

Прочитаем файл и сохраним в датасет `df`.

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.sample(10)

,calls,minutes,messages,mb_used,is_ultra
2583,41.0,250.28,6.0,14495.76,0
2531,124.0,786.97,73.0,27387.22,0
1116,101.0,694.20,41.0,9351.78,0
812,52.0,389.15,50.0,12774.10,0
2678,4.0,22.17,42.0,22635.73,0
126,129.0,848.86,0.0,13866.31,1
3010,66.0,522.07,13.0,21978.09,0
1325,96.0,612.26,0.0,17911.89,0
1781,36.0,285.92,109.0,19103.31,0
2608,68.0,391.55,31.0,24081.51,0


В датасете 3214 строк и 5 столбцов:
- `сalls` — количество звонков;
- `minutes` — суммарная длительность звонков в минутах;
- `messages` — количество sms-сообщений;
- `mb_used` — израсходованный интернет-трафик в Мб;
- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Преодобработка данных не требуется, пропусков нет. Переходим к разбивке датасета на выборки.

## Разбейте данные на выборки

Сначала разделим наш датасет на признаки (`calls`, `minutes`, `messages`, `mb_used`) и целевой признак (`is_ultra`).

In [5]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

Теперь разделим наш датасет на обучающую, тестовую и валидационную выборки в соотношении 60:20:20.

Сначала выделим обучающую и тестовую выборки в соотношении 80:20.

In [6]:
train_features, test_features, train_target, test_target = train_test_split(
    features, target, test_size=0.2, random_state=12345)

И выделим из полученной обучающей выборки еще 25 % для валидационной выборки. Такой процент используем, т.к. мы будем выделять валидационную выборку не из первоначального датасета, а из 80 % от него.

In [7]:
train_features, valid_features, train_target, valid_target = train_test_split(
    train_features, train_target, test_size=0.25, random_state=12345)

In [8]:
print('Размер обучающей выборки:', len(train_target))
print('Размер тестовой выборки:', len(test_target))
print('Размер валидационной выборки:', len(valid_target))

Размер обучающей выборки: 1928
Размер тестовой выборки: 643
Размер валидационной выборки: 643


Разделение датасета на 3 выборки прошло корректно. Приступаем к обучению моделей.

## Исследуйте модели

Поскольку перед нами стоит задача классификации, мы исследуем три модели машинного обучения:
- Дерево решений;
- Случайный лес;
- Логистическая регрессия.

### Дерево решений

In [9]:
#Обучим модель со стандартными параметрами
model = DecisionTreeClassifier(random_state=12345)
model.fit(train_features, train_target)
predictions = model.predict(valid_features)
accuracy_score(valid_target, predictions)

0.7122861586314152

Доля правильных ответов составляет всего 0.71, что очень мало. Проверим качество модели путем изменения гиперпараметра глубины дерева - `maх_depth` и гиперпараметра минимального числа объектов в листе - `min_samples_leaf`. Идеальные значения `min_samples_leaf` имеют тенденцию находиться в диапазоне от 1 до 20 для алгоритма CART (classification and regression trees – это аббревиатура, обозначающая методы классификации и регрессии с использованием дерева решений). Также параметр `min_samples_leaf` является наиболее ответственным за производительность конечных деревьев.

In [10]:
best_tree = None
best_result = 0
for depth in range(1, 10):
    for leaf in range(1, 20):
        tree_model = DecisionTreeClassifier(random_state=12345, max_depth=depth, min_samples_leaf=leaf)
        tree_model.fit(train_features, train_target)
        tree_predictions = tree_model.predict(valid_features)
        result = accuracy_score(valid_target, tree_predictions)
        if result > best_result:
            best_tree = tree_model
            best_result = result
            best_depth = depth
            best_leaf = leaf
print('Лучший результат:', best_result, 'с глубиной дерева', best_depth, 'и с числом объектов в листе', leaf)

Лучший результат: 0.7916018662519441 с глубиной дерева 7 и с числом объектов в листе 19


Получили результат с долей правильных ответов 0,79, при глубине дерева 7 и с 19 объектами в листе. Мы преодолели планку качества точности предсказания в 0,75 путем изменения гиперпараметров.

### Случайный лес

Рассмотрим теперь другую модель - случайный лес. Сделаем цикл с перебором количества деревьев от 1 до 30, чтобы выбрать лучшие гиперпараметры.

In [11]:
best_forest = None
best_result = 0
for est in range(1, 30):
    forest_model = RandomForestClassifier(random_state=12345, n_estimators=est)
    forest_model.fit(train_features, train_target)
    result = forest_model.score(valid_features, valid_target)
    if result > best_result:
        best_forest = forest_model
        best_result = result
        best_est = est
print('Лучший результат:', best_result, 'с количеством деревьев', best_est)

Лучший результат: 0.7931570762052877 с количеством деревьев 22


Как видим, получили тот же результат точности предсказания 0,79, как и в случае с деревом решений. При этом случайный лес работает с более низкой скоростью, чем дерево решений, поэтому предпочтение пока отдаем первой модели.

### Логистическая регрессия

И проверим точность нашей последней модели - логистическая регрессия. Это самая простая модель с малым количеством параметров и без склонности к переобучению.

In [12]:
reg_model = LogisticRegression(random_state=12345)
reg_model.fit(train_features, train_target)
result = reg_model.score(valid_features, valid_target)
print('Точность логистической регрессии:', result)

Точность логистической регрессии: 0.6967340590979783


Качество предсказаний логистической регрессии составляет всего 0,69, что очень мало и не преодолевает минимальный порог в 0,75.

### Вывод по разделу

Мы оценили качество трех обученных моделей на валидационной выборке и выяснили, что:
1. Качество дерева решений составляет 0,79.
2. Качество случайного леса также составляет 0,79.
3. Качество логистической регрессии составило 0,69 и она не преодолела минимальный порог точности.
4. У дерева решений и случайного леса одинаковый показатель качества, но при этом дерево решений работает быстрее, поэтому пока отдаем предпочтение данной модели.

## Проверьте модель на тестовой выборке

Проверим качество всех трех моделей на тестовой выборке, может здесь результаты будут отличаться от валидационной выборки.

In [13]:
#дерево решений
print('Качество дерева решений на тестовой выборке:', best_tree.score(test_features, test_target))

#случайный лес
print('Качество случайного леса на тестовой выборке:', best_forest.score(test_features, test_target))

#логистическая регрессия
print('Качество логистической регрессии на тестовой выборке:', reg_model.score(test_features, test_target))

Качество дерева решений на тестовой выборке: 0.7931570762052877
Качество случайного леса на тестовой выборке: 0.7838258164852255
Качество логистической регрессии на тестовой выборке: 0.702954898911353


<font size="5"><b>Вывод:</b> </font>

Итак, мы окончательно подтвердили предыдущие выводы и лучшей обученной моделью является <b>дерево решений с качеством предсказаний 0,79.</b>

## (бонус) Проверьте модели на адекватность

Для проверки модели на адекватность используем метод DummyClassifier.

DummyClassifier - это классификатор,который делает прогнозы,используя простые правила.

Стратегия использования для создания прогнозов:
- «Стратифицированный»: генерирует прогнозы с учетом распределения классов обучающей выборки.
- «Most_frequent»: всегда предсказывает наиболее частую метку в обучающем наборе.
- «Prior»: всегда предсказывает класс, который максимизирует предыдущий класс (например, «most_frequent»), а predict_proba возвращает предыдущий класс.
- «Uniform»: генерирует прогнозы равномерно в случайном порядке.
- «Константа»: всегда предсказывает постоянную метку, предоставленную пользователем. Это полезно для показателей, которые оценивают класс, не являющийся большинством

In [15]:
strategy = ['most_frequent', 'prior', 'stratified', 'uniform']
for index in strategy:
    dummy_clf = DummyClassifier(strategy=index, random_state=12345)
    dummy_clf.fit(train_features, train_target)
    score = dummy_clf.score(test_features, test_target)
    print('Точность для стратегии', index, ':', score)

Точность для стратегии most_frequent : 0.6951788491446346
Точность для стратегии prior : 0.6951788491446346
Точность для стратегии stratified : 0.5754276827371695
Точность для стратегии uniform : 0.48989113530326595


Как видим, при использовании самых простых способов обучения точность не достигает показателя нашего дерева решений, значит наша модель работает адекватно.